In [33]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# `pfynder` devel: v1

14.03.2021

**`pfynder`**

0. inputs: module name and function name to search for
1. find local of current python; effectively want `/usr/local/lib/pythonX.X/` where "pythonX.X" is the current version of the notebook
2. find the correct dir with the package's name, _ie_ "sklearn"
3. search through all `.py`-scripts using grep to find the function name
    - **Note** it could be in multiple modules, , filter to get those with "def"
4. output the results

In [1]:
!find /usr/local/lib/ -name "*sklearn" -type d | grep -e "python3.6"

/usr/local/lib/python3.6/dist-packages/sklearn


In [2]:
!whereis find

find: /usr/bin/find /usr/share/man/man1/find.1.gz /usr/share/info/find.info.gz


! ls /bin/find

In [3]:
import os

In [4]:
from subprocess import *

In [5]:
import platform

Get the current Python version (should be the kernel currently being used)

In [6]:
print(platform.python_version())
print(".".join(platform.python_version_tuple()[:2]))

3.6.9
3.6


In [7]:
import sys
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/sandm/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/sandm/.local/lib/python3.6/site-packages/IPython/extensions', '/home/sandm/.ipython']


In [8]:
def check_path(path):
    return (path != "") & \
        ((".zip" not in path) & \
         (".ipython" not in path) & \
         ("extensions" not in path)) & \
         ('packages' in path)

In [9]:
print("Paths to check")
checks = lambda p: f'\t{str(check_path(p)):5s}:' if check_path(p) else f'\t\t{str(check_path(p)):5s}:'
for p in sys.path:
    print(checks(p),p)

Paths to check
		False: /usr/lib/python36.zip
		False: /usr/lib/python3.6
		False: /usr/lib/python3.6/lib-dynload
		False: 
	True : /home/sandm/.local/lib/python3.6/site-packages
	True : /usr/local/lib/python3.6/dist-packages
	True : /usr/lib/python3/dist-packages
		False: /home/sandm/.local/lib/python3.6/site-packages/IPython/extensions
		False: /home/sandm/.ipython


In [10]:
!find /usr/local/lib/python3.6/dist-packages -name matplotlib -type d

/usr/local/lib/python3.6/dist-packages/matplotlib


In [262]:
import pkg_resources

packs = [str(l).split(' ')[0] for l in pkg_resources.working_set]

In [263]:
packs

['widgetsnbextension',
 'webencodings',
 'wcwidth',
 'typing-extensions',
 'typed-ast',
 'traitlets',
 'tornado',
 'testpath',
 'terminado',
 'tadasets',
 'six',
 'setuptools',
 'Send2Trash',
 'selenium',
 'scikit-tda',
 'ripser',
 'qtconsole',
 'pyzmq',
 'python-dateutil',
 'pyrsistent',
 'Pygments',
 'pybind11',
 'ptyprocess',
 'prompt-toolkit',
 'prometheus-client',
 'pip',
 'pickleshare',
 'phat',
 'pexpect',
 'persim',
 'parso',
 'pandocfilters',
 'notebook',
 'nbformat',
 'nbconvert',
 'mypy',
 'mypy-extensions',
 'mistune',
 'MarkupSafe',
 'kmapper',
 'jupyterlab',
 'jupyterlab-server',
 'jupyter',
 'jupyter-core',
 'jupyter-console',
 'jupyter-client',
 'jsonschema',
 'Jinja2',
 'jedi',
 'ipywidgets',
 'ipython',
 'ipython-genutils',
 'ipykernel',
 'hopcroftkarp',
 'entrypoints',
 'Deprecated',
 'defusedxml',
 'decorator',
 'Cython',
 'cechmate',
 'bleach',
 'backcall',
 'attrs',
 'zope.interface',
 'zipp',
 'xlrd',
 'xgboost',
 'wrapt',
 'Werkzeug',
 'w3lib',
 'virtualenv',
 '

In [46]:
def special_split(pack_path, astr):
    
    astr = astr.replace(pack_path+'/','')
    
    return astr.split("/")[1]

In [323]:
def pfynd(package_to_check, item_to_find, ismodule=False):
    
    pyvers = ".".join(platform.python_version_tuple()[:2])
    
    paths = [p for p in sys.path if check_path(p)]
    
    #print(paths)
    
    not_found = []
    
    for path in paths:
        
        cmds = ["find", path, "-type", "d"]
        
        # TODO: Need a way to ensure that the package name _is_ enforced
        # otherwise `grep` will just look for anything with `package_to_check`
        # in the STDOUT
        
        if (package_to_check != "") & (package_to_check != "*"):
            cmds.extend(["-name", package_to_check])
        
        # TODO: Need another way to do this as this way doesn't fully work
        if "*" in package_to_check: package_to_check="*"
            
        p1 = Popen(cmds, stdout=PIPE)
        p2 = Popen(["grep", f"python{pyvers}"], stdin=p1.stdout, stdout=PIPE)
        p3 = Popen(["grep",  "-v", "IPython"], stdin=p2.stdout, stdout=PIPE)
        p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
        output = p3.communicate()[0]

        dirs_to_search = [s for s in output.decode("utf-8").split('\n') if s != '']


        if dirs_to_search == []:
            not_found.append(f"\n\t\'{package_to_check}\' not found in {path}\n")
            continue

        #print(dirs_to_search)
        
        dir_to_search = dirs_to_search[0]

        #print(dir_to_search)

        if ismodule:
            p1 = Popen(["find", dir_to_search, "-type", "d", "-name", item_to_find], stdout=PIPE)
            #p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            #p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p1.communicate()[0]
            
            if (package_to_check != "") & (package_to_check != "*"):
                locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            else:
                locations = [s for s in output.decode("utf-8").split('\n') if s != '']
                

        else:
            p1 = Popen(["find", dir_to_search, "-type", "f", "-name", "*.py"], stdout=PIPE)
            p2 = Popen(["xargs", "grep", item_to_find], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            p2.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p3.communicate()[0]
            
            #print(output)

            #locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            #print(output)
            #print(dir_to_search)
            if (package_to_check != "") & (package_to_check != "*"):
                locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            else:
                locations = [s for s in output.decode("utf-8").split('\n') if s != '']            

            #print("locations:\n\t", '\n\t'.join(locations))

            dc = {}
            for l in locations:
                
                #print(l)
                # Check if the search package has been set
                #print("1. d2c", dir_to_search, "l",l)
                if (package_to_check == "") | (package_to_check == "*"):
                    pack_name = l.replace(dir_to_search+'/','').split('/')[0]
                    directory = dir_to_search + '/' + pack_name
                    #dir_to_search = '/'.join([dir_to_search, package_to_check])                
                    #print(dir_to_search)
                    l = l.split("/"+pack_name+"/")[1]
                    
                else:
                    pack_name = package_to_check
                    directory = dir_to_search
                    
                if pack_name not in packs: continue
                if pack_name not in dc.keys():
                    dc[pack_name] = {}
                    
                #print("2. d2c", directory, "l",l)
                
                #check if its a method, and then to which class it belongs
                if ('    ' in l) | ('        ' in l) | ('\t' in l): 
                    module, function = check_meth(directory, l)    
                else:
                    module = l.split('.py:')[0]
                    function = l.split('def ')[1].split("(")[0]                    
                
                if module in dc[pack_name].keys():
                    dc[pack_name][module].append(function)
                else:
                    dc[pack_name][module]= [function]

            if len(dc.keys()) != 0:
                #print(dc)
                for p_name in dc.keys():
                    print(f"{p_name}")

                    for k,v in dc[p_name].items():

                        print(f"\t{k}:")

                        for vv in v:
                            print(f"\t\t{vv}")
    
    show_fail = False
    if show_fail:
        print(not_found)
        
    return None

In [325]:
pfynd("plotly", "hist", False)

plotly
	validators.histogram2d._z.ZValidator:
		__init__
	validators.histogram2d._x.XValidator:
		__init__
	validators.histogram2d._uid.UidValidator:
		__init__
	validators.histogram2d._ids.IdsValidator:
		__init__
	validators.histogram2d._y.YValidator:
		__init__
	validators._histogram2d.Histogram2dValidator:
		__init__
	validators._histogram.HistogramValidator:
		__init__
	validators.histogram._x.XValidator:
		__init__
	validators.histogram._ysrc.YsrcValidator:
		__init__
	validators.histogram._ybins.YBinsValidator:
		__init__
	validators.histogram._xaxis.XAxisValidator:
		__init__
	validators.histogram._xbins.XBinsValidator:
		__init__
	validators.histogram._yaxis.YAxisValidator:
		__init__
	validators.histogram._text.TextValidator:
		__init__
	validators.histogram._uid.UidValidator:
		__init__
	validators.histogram._ids.IdsValidator:
		__init__
	validators.histogram._y.YValidator:
		__init__
	validators.histogram._name.NameValidator:
		__init__
	validators.histogram._xsrc.XsrcValid

In [306]:
def get_numb(numb, numbs):
    
    for n in numbs[::-1]:
        if numb < n: return n
    
    return numbs[-1]

def check_meth(dir_to_search, modulename):
    
    #print(l)
    module = modulename.split('.py:')[0]
    func = modulename.split('def ')[1].split("(")[0]
    #print(func)
    location = f"{dir_to_search}/{module}"+".py"

    # Check for all class names and their line numbers
    p1 = Popen(["grep", "-n", "^class", location], stdout=PIPE)
    output = p1.communicate()[0]                

    classdic = [s.split(':class ') for s in output.decode("utf-8").split('\n') if s != '']
    classdic = {int(pp[0]):pp[-1].split("(")[0] for pp in classdic}                    

    
    #print(classdic)
    classkeys = sorted(classdic.keys())

    #p1 = Popen(["find", dir_to_search, "-type", "f", "-name", "*.py"], stdout=PIPE)
    #p2 = Popen(["xargs", "grep", item_to_find], stdin=p1.stdout, stdout=PIPE)
    #p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)    
    
    p1 = Popen(["grep", "-n", func, location], stdout=PIPE)
    p2 = Popen(["grep", "def"], stdin=p1.stdout, stdout=PIPE)
    output = p2.communicate()[0]                

    methdic = [s.split(':')[0] for s in output.decode("utf-8").split('\n') if func in s]
    methdic = {int(pp):func for pp in methdic}
    methkeys = sorted(methdic.keys())
    #print(methdic)

    # Sometimes, these aren't classes, but actually nanmed, nested functions
    if classdic != {}:
        module += "."+classdic[[get_numb(kk, classkeys) for kk in methkeys ][0]]
        module = module.replace('/','.')
    
    return module, func

In [301]:
check_meth("/usr/local/lib/python3.6/dist-packages/nltk", "chunk/named_entity.py:    def _feature_detector(self, tokens, index, history):")

_feature_detector
{32: 'NEChunkParserTagger', 116: 'NEChunkParser'}
{57: '_feature_detector'}


('chunk.named_entity.NEChunkParser', '_feature_detector')

In [302]:
!grep histogram -n /usr/local/lib/python3.6/dist-packages/sklearn/cluster/tests/test_spectral.py

158:    def histogram(x, y, **kwargs):
163:    sp = SpectralClustering(n_clusters=2, affinity=histogram, random_state=0)


In [305]:
check_meth("/usr/local/lib/python3.6/dist-packages/sklearn", "cluster/tests/test_spectral.py:    def histogram(x, y, **kwargs):")

histogram
{}
{158: 'histogram'}


('cluster/tests/test_spectral', 'histogram')

In [249]:
import sys
sys

AttributeError: module 'sys' has no attribute 'packages'

```
/usr/local/lib/python3.6/dist-packages/tensorflow
tensorflow
	contrib/distributions/python/ops/sample_stats:
		auto_correlation
	contrib/metrics/python/ops/metric_ops:
		streaming_pearson_correlation
```

```
/usr/local/lib/python3.6/dist-packages/tensorflow
locations:
	 contrib/distributions/python/ops/sample_stats.py:def auto_correlation(
	contrib/metrics/python/ops/metric_ops.py:def streaming_pearson_correlation(predictions,
1. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/distributions/python/ops/sample_stats.py:def auto_correlation(
2. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/distributions/python/ops/sample_stats.py:def auto_correlation(
1. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/metrics/python/ops/metric_ops.py:def streaming_pearson_correlation(predictions,
2. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/metrics/python/ops/metric_ops.py:def streaming_pearson_correlation(predictions,
tensorflow
	contrib/distributions/python/ops/sample_stats:
		auto_correlation
	contrib/metrics/python/ops/metric_ops:
		streaming_pearson_correlation
```        

In [ ]:
2. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/distributions/python/ops/sample_stats.py:def auto_correlation(

2. d2c /usr/local/lib/python3.6/dist-packages/tensorflow l contrib/distributions/python/ops/sample_stats.py:def auto_correlation(


```
locations:
	 spatial/tests/test_distance.py:    def test_pdist_correlation_random(self):
	spatial/tests/test_distance.py:    def test_pdist_correlation_random_float32(self):
	spatial/tests/test_distance.py:    def test_pdist_correlation_random_nonC(self):
	spatial/tests/test_distance.py:    def test_pdist_correlation_iris(self):
	spatial/tests/test_distance.py:    def test_pdist_correlation_iris_float32(self):
	spatial/tests/test_distance.py:    def test_pdist_correlation_iris_nonC(self):
	spatial/tests/test_distance.py:    def test_correlation(self):
	spatial/distance.py:def _correlation_cdist_wrap(XA, XB, dm, **kwargs):
	spatial/distance.py:def _correlation_pdist_wrap(X, dm, **kwargs):
	spatial/distance.py:def correlation(u, v, w=None, centered=True):
scipy
	spatial.tests.test_distance.TestIsValidY:
		test_pdist_correlation_random
		test_pdist_correlation_random_float32
		test_pdist_correlation_random_nonC
		test_pdist_correlation_iris
		test_pdist_correlation_iris_float32
		test_pdist_correlation_iris_nonC
		test_correlation
	spatial/distance:
		_correlation_cdist_wrap
		_correlation_pdist_wrap
		correlation
```

In [ ]:
YES: /usr/local/lib/python3.6/dist-packages/scipy l spatial/tests/test_distance.py:    def test_pdist_correlation_random(self):
BAD: /usr/local/lib/python3.6/dist-packages l spatial/tests/test_distance.py:    def test_pdist_correlation_random(self):

In [117]:
l = 'spatial/tests/test_distance.py:    def test_pdist_correlation_random(self):'

In [118]:
dirto = '/usr/local/lib/python3.6/dist-packages/scipy'

In [109]:
pack_name = l.replace(dirto+'/','').split('/')[0]
dirto = '/'.join([dirto,ll])

In [120]:
print(module, function)

spatial.tests.test_distance.TestIsValidY test_pdist_correlation_random


In [49]:
special_split('/usr/local/lib/python3.6/dist-packages','/usr/local/lib/python3.6/dist-packages/junk1/junko')

'junko'

In [27]:
pfynd("matplotlib", "xlim", False)

locations:
	 axes/_base.py:    def get_xlim(self):
	axes/_base.py:    def set_xlim(self, left=None, right=None, emit=True, auto=False,
	pyplot.py:def xlim(*args, **kwargs):
	tests/test_skew.py:    def lower_xlim(self):
	tests/test_skew.py:    def upper_xlim(self):
	projections/geo.py:    def set_xlim(self, *args, **kwargs):
matplotlib
	axes._base._AxesBase:
		get_xlim
		set_xlim
	pyplot:
		xlim
	tests.test_skew.SkewXAxes:
		lower_xlim
		upper_xlim
	projections.geo.LambertAxes:
		set_xlim


In [ ]:
def pfynd(package_to_check, item_to_find, ismodule=False):
    
    pyvers = ".".join(platform.python_version_tuple()[:2])
    
    paths = [p for p in sys.path if check_path(p)]
    
    #print(paths)
    
    not_found = []
    
    for path in paths:
        
        cmds = ["find", path, "-type", "d"]
        
        if (package_to_check != "") | (package_to_check != "*"):
            cmds.extend(["-name", package_to_check])
        
        p1 = Popen(cmds, stdout=PIPE)
        p2 = Popen(["grep", f"python{pyvers}"], stdin=p1.stdout, stdout=PIPE)
        p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
        output = p2.communicate()[0]

        dirs_to_search = [s for s in output.decode("utf-8").split('\n') if s != '']

        if dirs_to_search == []:
            not_found.append(f"\n\t\'{package_to_check}\' not found in {path}\n")
            continue

        #print(dirs_to_search)
        
        dir_to_search = dirs_to_search[0]

        #print(dir_to_search)

        if ismodule:
            p1 = Popen(["find", dir_to_search, "-type", "d", "-name", item_to_find], stdout=PIPE)
            #p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            #p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p1.communicate()[0]
            
            if (package_to_check != "") & (package_to_check != "*"):
                locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            else:
                locations = [s for s in output.decode("utf-8").split('\n') if s != '']
                
            if len(locations) != 0:
                
                print(f"{package_to_check}")

                for l in locations:

                    print(f"\t{l}")

        else:
            p1 = Popen(["find", dir_to_search, "-type", "f", "-name", "*.py"], stdout=PIPE)
            p2 = Popen(["xargs", "grep", item_to_find], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            p2.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p3.communicate()[0]
            
            #print(output)
            print(dir_to_search)
            if (package_to_check != "") & (package_to_check != "*"):
                locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            else:
                locations = [s for s in output.decode("utf-8").split('\n') if s != '']
                
            

            print("locations:\n\t", '\n\t'.join(locations))

            dc = {}
            for l in locations:
                
                pack_name = l.replace(dir_to_search+'/','').split('/')[0]
                dir_to_search = '/'.join([dir_to_search,pack_name])
                
                #check if its a method, and then to which class it belongs
                if ('    ' in l) | ('\t' in l): 
                    
                    module, function = check_meth(dir_to_search, l)
                    
                    #print(module)
                    #try:
                    #    modpath = str_to_obj(package_to_check+'.'+module)
                    #    print(modpath)
                    #    print(isinstance(modpath, types.ClassType), hasattr(modpath, function))
                    #except:
                    #    pass
                    #         
                    
                else:
                    module = l.split('.py:')[0]
                    function = l.split('def ')[1].split("(")[0]                    
                
                if module in dc.keys():
                    dc[module].append(function)
                else:
                    dc[module]= [function]

            if len(dc.keys()) != 0:

                print(f"{package_to_check}")

                for k,v in dc.items():

                    print(f"\t{k}:")

                    for vv in v:
                        print(f"\t\t{vv}")
    
    show_fail = False
    if show_fail:
        print(not_found)
        
    return None